In [1]:
# Standard library imports
import json
from typing import List, Optional

# Third-party imports
from dotenv import load_dotenv
import tiktoken
from pprint import pprint

# llama-index imports
from llama_index.agent.openai import OpenAIAssistantAgent, OpenAIAgent

from llama_index.core.agent import ReActAgent

from llama_index.core.tools import FunctionTool, ToolMetadata
from llama_index.llms.openai import OpenAI

# Local imports
from ai_config_schema import LLM
from get_llm import get_llm, token_counter, get_llm_azure
from web_search import (
    get_page_content,
    process_search_results,
    generate_queries_search_engine,
)
from llama_index.core.agent.react import ReActChatFormatter
from react.custom_formatter import CustomReActChatFormatter

load_dotenv()


c:\Users\phi.nguyen\.conda\envs\neurond\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

# Agent with web tool


In [2]:
from react.agent_logger import log_step_output
import types
from react.ReActAgentCustom import ReActAgentCustom
from llama_index.core.agent import FunctionCallingAgent
from llama_index.core.tools import ToolOutput

In [3]:
model_name = "gpt-4o-mini"
# model_name = "claude-medium"


def run_agent(
    model_name: str,
    system_prompt: str,
    tools: Optional[List] = None,
    history: Optional[List] = None,
):
    if model_name in ["gpt-4o-mini", "gpt-4-turbo", "gpt-4"]:
        agent = OpenAIAgent.from_tools(
            tools=tools,
            system_prompt=system_prompt,
            chat_history=history,
            llm=get_llm_azure(),
            verbose=True,
            context=system_prompt,
        )

    else:
        agent = ReActAgent.from_tools(
            tools=tools,
            system_prompt=system_prompt,
            chat_history=history,
            llm=get_llm(model_name),
            verbose=True,
            context=system_prompt,
        )
    return agent

In [4]:
from system_prompt import DEFAULT_SYSTEM_PROMPT, DEFAULT_SYSTEM_PROMPT_OPTIMIZED
import datetime

DEFAULT_SYSTEM_PROMPT_WITH_TIME = DEFAULT_SYSTEM_PROMPT.format(
    date=str(datetime.datetime.now().date())
)
DEFAULT_SYSTEM_PROMPT_OPTIMIZED_WITH_TIME = DEFAULT_SYSTEM_PROMPT_OPTIMIZED.format(
    date=str(datetime.datetime.now().date())
)

In [6]:
# Configure tools
web_search_tool = FunctionTool.from_defaults(
    fn=process_search_results,
)

web_fetch_tool = FunctionTool.from_defaults(
    fn=get_page_content,
)

tools = [web_fetch_tool, web_search_tool]

In [11]:
DEFAULT_SYSTEM_PROMPT_WITH_TIME = DEFAULT_SYSTEM_PROMPT.format(
    date=str(datetime.datetime.now().date())
)
# DEFAULT_SYSTEM_PROMPT_WITH_TIME = DEFAULT_SYSTEM_PROMPT_OPTIMIZED.format(date = str(datetime.datetime.now().date()))

# agent.reset()
agent = run_agent(
    model_name=model_name,
    system_prompt=DEFAULT_SYSTEM_PROMPT_WITH_TIME,
    tools=tools,
    history="",
)
# agent.reset()

query = "kết quả trận T1 vs BLG mới nhất"
resp = agent.stream_chat(message=query)
response_text = ""


for token in resp.response_gen:
    response_text += token
    print(token, end="")


> Running step 64a0fe03-6978-46f3-8259-e0a48121b753. Step input: kết quả trận T1 vs BLG mới nhất
Thought: Tôi cần tìm kiếm thông tin mới nhất về kết quả trận đấu giữa T1 và BLG.
Action: web_search_tool
Action Input: {'input': 'T1 vs BLG latest match result November 2024'}
sub question: ['T1 vs BLG latest match result November 2024']
Observation: [{'url': 'https://www.gameleap.com/articles/worlds-2024-grand-final-t1-vs-blg-match-results'}, {'url': 'https://escorenews.com/en/lol/world-championship-2024/playoff/bilibili-gaming-vs-t1-571343'}, {'url': 'https://gol.gg/game/stats/62839/page-game/'}]
> Running step c5baaefe-4b53-4de7-995b-d22656c53f4e. Step input: None
Thought: Tôi đã tìm thấy một số liên kết có thể chứa thông tin về kết quả trận đấu giữa T1 và BLG. Tôi sẽ lấy thông tin từ các liên kết này để cung cấp kết quả chính xác.
Action: web_fetch_tool
Action Input: {'input': 'https://www.gameleap.com/articles/worlds-2024-grand-final-t1-vs-blg-match-results'}
Observation: Worlds 2024 G

In [12]:
query = "tôi vừa hỏi gì?"
resp = agent.stream_chat(message=query)
response_text = ""


for token in resp.response_gen:
    response_text += token
    print(token, end="")


> Running step beb80b08-fa5c-4a9d-a5e9-2876bc254362. Step input: tôi vừa hỏi gì?
Bạn đãBạn đã hỏi về kết quả trận đấu giữa T1 và BLG mới nhất.

In [13]:
query = "Tổng bí thư là ai?"
resp = agent.stream_chat(message=query)
response_text = ""


for token in resp.response_gen:
    response_text += token
    print(token, end="")


> Running step 929ff556-6d38-4c02-aae3-4ea4a7950797. Step input: Tổng bí thư là ai?
TổngTổng bí thư của Đảng Cộng sản Việt Nam là Nguyễn Phú Trọng. Ông đã giữ chức vụ này từ tháng 1 năm 2011 và là một trong những nhân vật quan trọng nhất trong chính trị Việt Nam.

In [ ]:
print(response_text)

# StreamOpenAIAgent

In [18]:
# Configure tools
import datetime

web_search_tool = FunctionTool.from_defaults(
    fn=process_search_results,
    name="web_search_tool",
    description=(
        f"Used to retrieve information about up-to-date information in {str(datetime.datetime.now().date())}, \
            website or the information out of LLM's knowledge."
    ),
)

web_fetch_tool = FunctionTool.from_defaults(
    fn=get_page_content,
    name="web_fetch_tool",
    description="Used to fetch information of specific url",
)

tools = [web_fetch_tool, web_search_tool]


In [19]:
from react.StreamOpenAIAgent import StreamOpenAIAgent
from system_prompt import DEFAULT_SYSTEM_PROMPT
from llama_index.core.chat_engine.types import StreamingAgentChatResponse

DEFAULT_SYSTEM_PROMPT_WITH_TIME = DEFAULT_SYSTEM_PROMPT.format(
    date=str(datetime.datetime.now().date())
)
agent = StreamOpenAIAgent.from_tools(
    tools=tools,
    system_prompt=DEFAULT_SYSTEM_PROMPT_WITH_TIME,
    chat_history="",
    llm=get_llm_azure(),
    verbose=False,
)


In [20]:
input = "thế còn chủ tịch nước là ai?"
from llama_index.core.chat_engine.types import StreamingAgentChatResponse

resp_gen = agent.stream_chat_custom(message=input)

for step_output in resp_gen:
    # Check if the step_output is StreamingAgentChatResponse -> do not need to log, return final response
    if isinstance(step_output, StreamingAgentChatResponse):
        resp = step_output
    # Log intermediate steps (reasoning steps)
    else:
        log_data = {
            "conversation_id": "fake",
            "reasoning_data": {
                "input": step_output.output.sources[-1].raw_input[
                    "kwargs"
                ],  # Get latest input of tool
                "tool_name": step_output.output.sources[
                    -1
                ].tool_name,  # Get latest tool call
                # "output": str(step_output.output),
            },
        }

        print(f"```reasoning_log\n{log_data}\n```")

response = ""
for token in resp.response_gen:
    response += token
    print(token, end="")


Exception in thread Thread-4:
Traceback (most recent call last):
  File "c:\Users\phi.nguyen\.conda\envs\test\lib\threading.py", line 980, in _bootstrap_inner
    self.run()
  File "c:\Users\phi.nguyen\.conda\envs\test\lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\phi.nguyen\.conda\envs\test\lib\threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Users\phi.nguyen\.conda\envs\test\lib\site-packages\llama_index\core\instrumentation\dispatcher.py", line 311, in wrapper
    result = func(*args, **kwargs)
  File "c:\Users\phi.nguyen\.conda\envs\test\lib\site-packages\llama_index\core\chat_engine\types.py", line 177, in write_response_to_history
    for chat in self.chat_stream:
  File "c:\Users\phi.nguyen\.conda\envs\test\lib\site-packages\llama_index\core\llms\callbacks.py", line 186, in wrapped_gen
    for x in f_return_val:
  File "c:\Users\phi.nguyen\.conda\envs\test\lib\site-packa

Got final result! 


AuthenticationError: Error code: 401 - {'statusCode': 401, 'message': 'Unauthorized. Access token is missing, invalid, audience is incorrect (https://cognitiveservices.azure.com), or have expired.'}

# ReAct Workflow

In [20]:
# Configure tools
import datetime
from react.ReActAgentWR import FinalStep, ReasoningStep
import datetime

# from react.ReActAgentW import ReActAgentW
from react.ReActAgentWR import ReActAgentW
from system_prompt import DEFAULT_SYSTEM_PROMPT

DEFAULT_SYSTEM_PROMPT_WITH_TIME = DEFAULT_SYSTEM_PROMPT.format(
    date=str(datetime.datetime.now().date())
)

web_search_tool = FunctionTool.from_defaults(
    fn=process_search_results,
    name="web_search_tool",
    description=(
        f"Used to retrieve information about up-to-date information in {str(datetime.datetime.now().date())}, \
            website or the information out of LLM's knowledge."
    ),
)

web_fetch_tool = FunctionTool.from_defaults(
    fn=get_page_content,
    name="web_fetch_tool",
    description="Used to fetch information of specific url",
)

tools = [web_fetch_tool, web_search_tool]


In [23]:
from react.ReActAgentWR import FinalStep, ReasoningStep
import datetime

# from react.ReActAgentW import ReActAgentW
from react.ReActAgentWR import ReActAgentW
from system_prompt import DEFAULT_SYSTEM_PROMPT

DEFAULT_SYSTEM_PROMPT_WITH_TIME = DEFAULT_SYSTEM_PROMPT.format(
    date=str(datetime.datetime.now().date())
)
model_name = "gpt-4o-mini"
# model_name = "claude-medium"
# model_name = "gemini-pro"


def run_agent(
    model_name: str,
    system_prompt: str,
    tools: Optional[List] = None,
    history: Optional[List] = None,
):
    if model_name in ["gpt-4o-mini", "gpt-4-turbo", "gpt-4"]:
        agent = ReActAgentW(
            tools=tools,
            extra_context=system_prompt,
            chat_history=history,
            llm=get_llm_azure(),
            verbose=False,
            timeout=60,
        )

    else:
        agent = ReActAgentW(
            tools=tools,
            extra_context=system_prompt,
            chat_history=history,
            llm=get_llm(model_name),
            verbose=False,
            timeout=60,
        )

    return agent


# agent.reset()
agent = run_agent(
    model_name=model_name,
    system_prompt=DEFAULT_SYSTEM_PROMPT_WITH_TIME,
    tools=tools,
    history=None,
)


async def stream_chat_gen(agent, input):
    handler = agent.run(input=input, timeout=60)
    async for ev in handler.stream_events():
        if isinstance(ev, ReasoningStep):
            print("#Reasoning:")
            yield (ev.data)
        elif isinstance(ev, FinalStep):
            yield (ev.delta)

In [24]:
input = "kết quả trận T1 vs BLG mới nhất"
handler = agent.run(input=input, timeout=60)

async for ev in handler.stream_events():
    if isinstance(ev, ReasoningStep):
        print("#Reasoning:")
        print(ev.data)
    elif isinstance(ev, FinalStep):
        print(ev.delta, end="")


['', 'Thought', ':', ' The', ' current', ' language', ' of', ' the', ' user', ' is', ':', ' Vietnamese', '.', ' I', ' need', ' to', ' use', ' a', ' tool', ' to', ' help', ' me', ' answer', ' the', ' question', '.\n', 'Action', ':', ' web', '_search', '_tool', '\n', 'Action', ' Input', ':', ' {"', 'input', '":', ' "', 'k', 'ết', ' quả', ' trận', ' T', '1', ' vs', ' BL', 'G', ' mới', ' nhất', '"}', '']
#Reasoning:
tool_id='fake' tool_name='web_search_tool' tool_kwargs={'input': 'kết quả trận T1 vs BLG mới nhất'}
sub question: ['kết quả trận T1 vs BLG mới nhất']
['', 'Thought', ':', ' I', ' have', ' found', ' some', ' sources', ' that', ' may', ' contain', ' the', ' latest', ' results', ' for', ' the', ' match', ' between', ' T', '1', ' and', ' BL', 'G', '.', ' I', ' will', ' check', ' the', ' most', ' relevant', ' one', ' for', ' the', ' latest', ' results', '.\n', 'Action', ':', ' web', '_fetch', '_tool', '\n', 'Action', ' Input', ':', " {'", 'input', "':", " '", 'https', '://', 'web', 

In [25]:
input = "tôi vừa hỏi gì?"
handler = agent.run(input=input, timeout=60)

async for ev in handler.stream_events():
    if isinstance(ev, ReasoningStep):
        print("#Reasoning:")
        print(ev.data)
    elif isinstance(ev, FinalStep):
        print(ev.delta, end="")


['', 'Bạn', ' đã', ' hỏi', ' về', ' kết', ' quả', ' trận', ' đấu', ' giữa', ' T', '1', ' và', ' BL', 'G', ' mới', ' nhất', '.', '']

Bạn đã hỏi về kết quả trận đấu giữa T1 và BLG mới nhất.

In [26]:
input = "Tổng bí thư là ai?"
handler = agent.run(input=input, timeout=60)

async for ev in handler.stream_events():
    if isinstance(ev, ReasoningStep):
        print("#Reasoning:")
        print(ev.data)
    elif isinstance(ev, FinalStep):
        print(ev.delta, end="")


['', 'Thought', ':', ' The', ' current', ' language', ' of', ' the', ' user', ' is', ':', ' Vietnamese', '.', ' I', ' need', ' to', ' use', ' a', ' tool', ' to', ' help', ' me', ' answer', ' the', ' question', '.\n', 'Action', ':', ' web', '_search', '_tool', '\n', 'Action', ' Input', ':', ' {"', 'input', '":', ' "', 'T', 'ổng', ' bí', ' thư', ' hiện', ' nay', ' là', ' ai', '?"', '}', '']
#Reasoning:
tool_id='fake' tool_name='web_search_tool' tool_kwargs={'input': 'Tổng bí thư hiện nay là ai?'}
sub question: ['Tổng bí thư hiện nay là ai?']
['', 'Thought', ':', ' I', ' have', ' found', ' relevant', ' information', ' about', ' the', ' current', ' General', ' Secretary', '.', ' I', ' will', ' extract', ' the', ' necessary', ' details', ' from', ' the', ' sources', '.\n', 'Action', ':', ' web', '_fetch', '_tool', '\n', 'Action', ' Input', ':', " {'", 'input', "':", " '", 'https', '://', 'vi', '.wikipedia', '.org', '/wiki', '/T', 'ổng', '_B', 'í', '_th', 'ư', '_B', 'an', '_Ch', 'ấp', '_h', 

In [27]:
input = "tôi vừa hỏi gì ở câu trước và câu trước nữa?"
handler = agent.run(input=input, timeout=60)

async for ev in handler.stream_events():
    if isinstance(ev, ReasoningStep):
        print("#Reasoning:")
        print(ev.data)
    elif isinstance(ev, FinalStep):
        print(ev.delta, end="")


['', 'Bạn', ' đã', ' hỏi', ' về', ' kết', ' quả', ' trận', ' đấu', ' giữa', ' T', '1', ' và', ' BL', 'G', ',', ' và', ' sau', ' đó', ' bạn', ' hỏi', ' về', ' ai', ' là', ' Tổng', ' Bí', ' thư', '.', '']

Bạn đã hỏi về kết quả trận đấu giữa T1 và BLG, và sau đó bạn hỏi về ai là Tổng Bí thư.

In [28]:
str(handler)


"{'response': 'Bạn đã hỏi về kết quả trận đấu giữa T1 và BLG, và sau đó bạn hỏi về ai là Tổng Bí thư.', 'sources': [], 'reasoning': []}"

In [ ]:
"""
Ai hiện đang đảm nhiệm vai trò COO và CTO tại Orient Software?
Cho biết tên của Giám đốc Điều hành (COO) và Giám đốc Công nghệ (CTO) của Orient Software?
Hiện tại, ai là COO và CTO của công ty Orient Software?
Người giữ chức vụ COO và CTO của Orient Software là ai?
Hãy cung cấp thông tin về COO và CTO của Orient Software.s
"""

In [11]:
a = "\nhello"
print(a.strip().find("3"))

-1


# PhiAgent

In [1]:
from react.PhiAgent import PhiAgent
import datetime
from typing import List, Optional
from system_prompt import DEFAULT_SYSTEM_PROMPT
from llama_index.core.tools import FunctionTool
from web_search import (
    get_page_content,
    process_search_results,
)
from react.PhiAgent import ResponseStreamEvent, ToolStreamEvent
from get_llm import get_llm, get_llm_azure

DEFAULT_SYSTEM_PROMPT_WITH_TIME = DEFAULT_SYSTEM_PROMPT.format(
    date=str(datetime.datetime.now().date())
)

web_search_tool = FunctionTool.from_defaults(
    fn=process_search_results,
    name="web_search_tool",
    description=(
        f"Used to retrieve information about up-to-date information in {str(datetime.datetime.now().date())}, \
            website or the information out of LLM's knowledge."
    ),
)

web_fetch_tool = FunctionTool.from_defaults(
    fn=get_page_content,
    name="web_fetch_tool",
    description="Used to fetch information of specific url",
)

tools = [web_fetch_tool, web_search_tool]


c:\Users\phi.nguyen\.conda\envs\test\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
DEFAULT_SYSTEM_PROMPT_WITH_TIME = DEFAULT_SYSTEM_PROMPT.format(
    date=str(datetime.datetime.now().date())
)
model_name = "gpt-4o-mini"
# model_name = "claude-medium"
# model_name = "gemini-pro"


def run_agent(
    model_name: str,
    system_prompt: str,
    tools: Optional[List] = None,
    history: Optional[List] = None,
):
    if model_name in ["gpt-4o-mini", "gpt-4-turbo", "gpt-4"]:
        agent = PhiAgent(
            tools=tools,
            system_prompt=system_prompt,
            # chat_history=history,
            llm=get_llm_azure(),
            verbose=False,
            timeout=60,
        )

    else:
        agent = PhiAgent(
            tools=tools,
            system_prompt=system_prompt,
            # chat_history=history,
            llm=get_llm(model_name),
            verbose=False,
            timeout=60,
        )

    return agent


# agent.reset()
agent = run_agent(
    model_name=model_name,
    system_prompt=DEFAULT_SYSTEM_PROMPT_WITH_TIME,
    tools=tools,
    history=None,
)


async def stream_chat_gen(agent, input):
    handler = agent.run(input=input, timeout=60)
    async for ev in handler.stream_events():
        if isinstance(ev, ToolStreamEvent):
            print("#Reasoning:")
            yield (ev.text)
        elif isinstance(ev, ResponseStreamEvent):
            yield (ev.text)

In [3]:
input = "kết quả trận T1 vs BLG mới nhất"
handler = agent.run(input=input, timeout=60)

async for ev in handler.stream_events():
    if isinstance(ev, ToolStreamEvent):
        print("#Reasoning:")
        print(ev.text)
    elif isinstance(ev, ResponseStreamEvent):
        print(ev.text, end="")


#Reasoning:
[ToolSelection(tool_id='call_PuYQUc1RO5bMcfmKBWwM0iiZ', tool_name='web_search_tool', tool_kwargs={'input': 'T1 vs BLG latest match result November 2024'})]
sub question: ['T1 vs BLG latest match result November 2024']
#Reasoning:
[ToolSelection(tool_id='call_w5PEESBCKuVbVQsQOGAo2Nj2', tool_name='web_fetch_tool', tool_kwargs={'input': 'https://www.gameleap.com/articles/worlds-2024-grand-final-t1-vs-blg-match-results'}), ToolSelection(tool_id='call_1YCGNVGGvlYS3KMwIbn9wcDT', tool_name='web_fetch_tool', tool_kwargs={'input': 'https://www.sportskeeda.com/esports/t1-vs-bilibili-gaming-league-legends-worlds-2024-grand-final-prediction-livestream-details'}), ToolSelection(tool_id='call_wqUxe85xg9EZOZ3OqeOiA1s7', tool_name='web_fetch_tool', tool_kwargs={'input': 'https://scores24.live/en/lol/m-12-05-2024-bilibili-gaming-t1'})]
In the recent match between T1 and Bilibili Gaming (BLG) during the **2024 League of Legends World Championship Grand Final**, T1 emerged victorious with a f

In [14]:
input = "tôi vừa hỏi gì?"
handler = agent.run(input=input, timeout=60)

async for ev in handler.stream_events():
    if isinstance(ev, ToolStreamEvent):
        print("#Reasoning:")
        print(ev.text)
    elif isinstance(ev, ResponseStreamEvent):
        print(ev.text, end="")


RawContentBlockDeltaEvent(delta=TextDelta(text='Để', type='text_delta'), index=0, type='content_block_delta')


In [15]:
input = "Thế còn chủ tịch nước?"
handler = agent.run(input=input, timeout=60)

async for ev in handler.stream_events():
    if isinstance(ev, ToolStreamEvent):
        print("#Reasoning:")
        print(ev.text)
    elif isinstance(ev, ResponseStreamEvent):
        print(ev.text, end="")


RawContentBlockDeltaEvent(delta=TextDelta(text='Để', type='text_delta'), index=0, type='content_block_delta')
